In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Currently just import to register the accessors.
# This seems a common pattern for other libraries.
import pdcomfi.accessors

In [ ]:
model = gp.Model()
series = pd.Series(
    index=pd.RangeIndex(5, 10),
    data=np.random.random(5),
)
series

In [ ]:
# From an existing series, create variables using
# the index accessor. Names are based on index values.
x = series.index.grb.addVars(model, name="x")
model.update()
x

In [ ]:
# Can pass individual attributes, but this will
# be cleaner through df attributes.
y = series.index.grb.addVars(model, ub=5.0, name="y")
model.update()

In [ ]:
# Nothing special for arithmetic operations,
# pandas still handles this itself.
x + y

In [ ]:
# Use series accessors to read attributes.
y.grb.ub

In [ ]:
# Use series accessors to set attributes.
y.grb.lb = series
model.update()
y.grb.lb

In [ ]:
# Things are a bit cleaner with a dataframe.
df = pd.DataFrame({
    "a": list(range(5)),
    "b": list(range(5, 10)),
    "c": np.random.random(5),
})
df

In [ ]:
# addVars: append a column of new variables
# use selected columns for naming
df1 = df.grb.addVars(model, name="w", index=["a", "b"])
model.update()
df

In [ ]:
# addVars using multiindex for naming
# use a column to set objective coefficients
model.setObjective(0)
df2 = df1.set_index(["a", "b"]).grb.addVars(model, name="z", obj="c")
model.update()
df2

In [ ]:
model.getObjective()

In [ ]:
# We can't naturally do overloaded comparisons with just the accessors.
# Pandas forces a boolean cast ... more on that later.
df2['w'] + df2['z'] >= 1

In [ ]:
# Proposed API: use .grb.addConstrs to create a constraint between two columns or values.
df3 = df2.grb.addConstrs(model, lhs="w", sense=GRB.GREATER_EQUAL, rhs=0.5, name="w_lower")
model.update()
df3

In [ ]:
# Alternative df.eval/df.query style
# Simplifies cases where we would need an intermediate
# column to capture 'w + z'
df4 = df3.grb.addConstrs(model, "w + z >= c", name="wz")
model.update()
df4

In [ ]:
model.optimize()

In [ ]:
# Use series accessor to extract results
df4['w'].grb.X

In [ ]:
df4['z'].grb.X

In [ ]:
df4['wz'].grb.Slack

In [ ]:
df4['w_lower'].grb.Slack